# External Database Data Generation

The mentor indicated that the given notebook in the starter code is not supposed to be used and I should generate my own support tickets to be used as input of the solution.

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from typing import List
from pydantic import BaseModel, Field
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Literal, TypedDict, Annotated
from langchain.agents import create_agent

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_base_url = "https://openai.vocareum.com/v1"

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    base_url="https://openai.vocareum.com/v1",
    api_key=OPENAI_API_KEY,
)

Sampling Paramters for Tickets

In [2]:
num_account = 5
num_user = 10
num_ticket = 5
num_msg = num_ticket
num_expected_tag_msg = round(num_msg * 0.8)
num_unexpected_tag_msg = num_msg - num_expected_tag_msg
expected_tags = ['access', 'attendance', 'benefits', 'billing', 'booking', 'cancelation', 'escalation', 'events', 'login', 'password', 'pause', 'pricing', 'reservation', 'subscription', 'support']
unexpected_tags = ['weather', 'stock price', 'dietary', 'health advice']
tags = expected_tags + unexpected_tags

In [3]:
# Random Time Stamp Function

import datetime
import random

def generate_random_timestamps(count=5, start_year=2024, end_year=2025):
    """
    Generates a list of random Unix timestamps between two dates.
    
    Args:
        count (int): The number of timestamps to generate.
        start_year (int): The starting year for the random range.
        end_year (int): The ending year for the random range.
        
    Returns:
        list: A list of integers, where each integer is a Unix timestamp.
    """
    # 1. Define the start and end dates
    start_dt = datetime.datetime(start_year, 1, 1, 0, 0, 0)
    end_dt = datetime.datetime(end_year, 12, 31, 23, 59, 59)
    
    # 2. Convert the datetime objects to their Unix timestamp (integer seconds)
    #    This gives us the range of numbers (seconds) to pick from.
    start_unix = int(start_dt.timestamp())
    end_unix = int(end_dt.timestamp())
    
    # 3. Generate the list of random integers
    random_timestamps = []
    for _ in range(count):
        random_ts = random.randint(start_unix, end_unix)
        random_timestamps.append(random_ts)
        
    return random_timestamps

# Create Tables

Account

In [4]:
class AccountSamples(BaseModel):
    """Structured response for account table"""
    name: List[str] = Field(description="List of first name and last name")

account_prompt_template = """You are a company naming expert. Please come up with {num_account} company names."""

account_prompt = PromptTemplate(
    template = account_prompt_template,
    input_variables = ["num_account"],
    ).invoke({"num_account": num_account}).to_messages()

account = llm.with_structured_output(AccountSamples).invoke(account_prompt).model_dump()['name']
account_table = pd.DataFrame({"name": account})
account_table['id'] = account_table.index
account_table_ls = account_table.to_dict(orient='records')
account_table

,name,id
0,InnovateX Solutions,0
1,EcoSphere Ventures,1
2,QuantumLeap Technologies,2
3,Harmony Health Group,3
4,StellarWave Enterprises,4


User

In [5]:
class SingleUser(BaseModel):
    """Structured response for a single user"""
    email: str = Field(description="User's email address.")
    display_name: str = Field(description="User's name with first and last name.")
    account_id: int = Field(description="Account ID")

class UserSamples(BaseModel):
    """Structured response for a user list"""
    user_list: List[SingleUser]

user_prompt_template = """
    You are a people naming expert. Please come up with {num_user} names with both first and last name.
    Randomly choose a company for each user from the account list and based on the name of the company,
    create an email address for this person.  Some companies should have more users than others.

    All users from the same company must have the same domain name on their email address.

    The account ID is the ID in the provided table, the ID and account name must match.

    List of account (companies): {account_table}"""

user_prompt = PromptTemplate(
    template = user_prompt_template,
    input_variables = ["num_user","account_table"],
    ).invoke({"num_user": num_user, "account_table": account_table}).to_messages()

user_ls = llm.with_structured_output(UserSamples).invoke(user_prompt).model_dump()['user_list']
user_table = pd.DataFrame(user_ls)
user_table['id'] = user_table.index
user_table

,email,display_name,account_id,id
0,john.doe@innovatexsolutions.com,John Doe,0,0
1,jane.smith@innovatexsolutions.com,Jane Smith,0,1
2,michael.jones@ecospheraventures.com,Michael Jones,1,2
3,emily.brown@ecospheraventures.com,Emily Brown,1,3
4,william.taylor@quantumleaptech.com,William Taylor,2,4
5,olivia.martin@harmonyhealthgroup.com,Olivia Martin,3,5
6,sophia.wilson@harmonyhealthgroup.com,Sophia Wilson,3,6
7,james.moore@stellarwaveenterprises.com,James Moore,4,7
8,isabella.thomas@stellarwaveenterprises.com,Isabella Thomas,4,8
9,liam.jackson@stellarwaveenterprises.com,Liam Jackson,4,9


In [6]:
class SingleTicket(BaseModel):
    """Structured response for a ticket"""
    account_id: Annotated[int, Field(description="Account ID")]
    created_by_user_id: Annotated[int, Field(description="User ID")]
    # status: Annotated[Literal['open','pending','closed'], Field(description="Status of the ticket")]
    priority: Annotated[Literal['low','normal','high','urgent'], Field(description="Ticket priority")]
    channel: Annotated[Literal['email','chat','web','sms','social','phone'], Field(description="Ticket priority")]
    # created_at: Annotated[str, Field(description="Timestamp of the corresponding ticket")]

class TicketSamples(BaseModel):
    """Structured response of ticket list"""
    user_list: List[SingleTicket]

ticket_prompt_template = """
    You are a IT ticket generation emulator. Generate {num_ticket} tickets with random selection of
    priority ('low','normal','high','urgent'), channel ('email','chat','web','sms','social','phone').

    The account_id and id is based on this table:
    {user_table}
    """

ticket_prompt = PromptTemplate(
    template = ticket_prompt_template,
    input_variables = ["num_ticket","user_table"],
    ).invoke({"num_ticket": num_ticket, "user_table": user_table}).to_messages()

ticket_ls = llm.with_structured_output(TicketSamples).invoke(ticket_prompt).model_dump()['user_list']
ticket_table = pd.DataFrame(ticket_ls)
ticket_table['id'] = ticket_table.index
ts_format = "%Y-%m-%d %H:%M:%S"
timestamps = generate_random_timestamps(count=5, start_year=2024, end_year=2025)
ticket_table['create_at'] = [ datetime.datetime.fromtimestamp(ts).strftime(ts_format) for ts in timestamps ]
ticket_table['status'] = 'open'
ticket_table

,account_id,created_by_user_id,priority,channel,id,create_at,status
0,0,0,high,email,0,2025-01-15 03:30:52,open
1,1,3,normal,chat,1,2024-07-28 23:33:05,open
2,2,4,urgent,web,2,2025-06-23 22:38:14,open
3,3,5,low,sms,3,2025-12-23 18:55:29,open
4,4,8,normal,phone,4,2024-06-28 21:59:10,open


In [7]:
class SingleMessage(BaseModel):
    """Structured response for a message"""
    # role: str = Annotated[Literal['user','agent','system'], Field(description="The type of role for this message.")]
    body: Annotated[str, Field(description="The ticket body message content.")]
    sent_at: Annotated[str, Field(description="Date and time.")]
    ticket_id: Annotated[int, Field(description="Ticket ID")]
    author_user_id: Annotated[int, Field(description="User ID")]
    tags_used: Annotated[List[str], Field(description="List of tags used for creating this message")]

class MessageSamples(BaseModel):
    """Structured response of a message list"""
    msg_list: List[SingleMessage]

msg_prompt_template = """
    You are a IT ticket generation emulator focusing on generating the actual message of users. 
    
    Step 1: Generate text message `body` with each message less than 200 words, and make occasional minor typos in the message:
        Generate {num_expected_tag_msg} messages body based on the selection of one to multiple of the following topics {expected_tags}.
        Generate {num_unexpected_tag_msg} messages with at least one topic from {unexpected_tags} as main theme and zero to multiple topics from {expected_tags}.  

    Step 2: For the author_user_id (`created_by_user_id` column in the table) and ticket_id (`id` column in the table), select one of the ticket from the ticket table:
    {ticket_table}

    Step 3: The the `send_at` time stamp, use the `create_at` timestamp from the above table and add at least 1 hours to 20 days.
    """

msg_prompt = PromptTemplate(
    template = msg_prompt_template,
    input_variables = ["num_expected_tag_msg","num_unexpected_tag_msg","expected_tags","unexpected_tags","ticket_table"],
    ).invoke({"num_expected_tag_msg": num_expected_tag_msg, 
              "num_unexpected_tag_msg": num_unexpected_tag_msg, 
              "expected_tags": expected_tags, 
              "unexpected_tags": unexpected_tags, 
              "ticket_table": ticket_table}).to_messages()

msg_ls = llm.with_structured_output(MessageSamples).invoke(msg_prompt).model_dump()['msg_list']
msg_table = pd.DataFrame(msg_ls)
msg_table['id'] = msg_table.index
msg_table['role'] = 'user'
msg_table


,body,sent_at,ticket_id,author_user_id,tags_used,id,role
0,"Hi there, I'm having trouble with my login cre...",2025-01-16 04:30:52,0,0,"[login, password, access]",0,user
1,"Hello, I need to cancel my reservation for the...",2024-07-29 00:33:05,1,3,"[cancelation, reservation, events]",1,user
2,"Hi, I have a question about my subscription. I...",2025-06-24 23:38:14,2,4,"[subscription, billing]",2,user
3,"Good day, I'm experiencing issues with my atte...",2025-12-24 19:55:29,3,5,"[attendance, support]",3,user
4,"Hi, I was wondering if you could provide some ...",2024-06-29 22:59:10,4,8,"[health advice, dietary, access]",4,user


In [8]:
class SingleMeta(BaseModel):
    ticket_id: Annotated[int, Field(description="Ticket ID")]
    key: Annotated[str, Field(description="")]
    value: Annotated[str, Field(description="")]

class MetaSamples(BaseModel):
    """Structured response of a metadata list"""
    meta_list: List[SingleMessage]

In [9]:
class SingleKnowledge(BaseModel):
    title: Annotated[str, Field(description="Knowledge title")]
    body: Annotated[str, Field(description="Knowledege body content")]
    source: Annotated[str, Field(description="url/path/system")]

class MetaSamples(BaseModel):
    """Structured response of a metadata list"""
    knowledge_list: List[SingleKnowledge]

# Create Database

In [ ]:
from sqlalchemy import create_engine
from utils import reset_db, get_session, model_to_dict

In [ ]:
ticket_db = "data/external/ticket.db"
reset_db(ticket_db)
engine = create_engine(f"sqlite:///{ticket_db}", echo=False)
ticket.Base.metadata.create_all(engine)